# Day 18: Boiling Boulders

[https://adventofcode.com/2022/day/18](https://adventofcode.com/2022/day/18)

## Description

### Part One

You and the elephants finally reach fresh air. You've emerged near the base of a large volcano that seems to be actively erupting! Fortunately, the lava seems to be flowing away from you and toward the ocean.

Bits of lava are still being ejected toward you, so you're sheltering in the cavern exit a little longer. Outside the cave, you can see the lava landing in a pond and hear it loudly hissing as it solidifies.

Depending on the specific compounds in the lava and speed at which it cools, it might be forming [obsidian](https://en.wikipedia.org/wiki/Obsidian)! The cooling rate should be based on the surface area of the lava droplets, so you take a quick scan of a droplet as it flies past you (your puzzle input).

Because of how quickly the lava is moving, the scan isn't very good; its resolution is quite low and, as a result, it approximates the shape of the lava droplet with _1x1x1 <span title="Unfortunately, you forgot your flint and steel in another dimension.">cubes</span> on a 3D grid_, each given as its `x,y,z` position.

To approximate the surface area, count the number of sides of each cube that are not immediately connected to another cube. So, if your scan were only two adjacent cubes like `1,1,1` and `2,1,1`, each cube would have a single side covered and five sides exposed, a total surface area of _`10`_ sides.

Here's a larger example:

    2,2,2
    1,2,2
    3,2,2
    2,1,2
    2,3,2
    2,2,1
    2,2,3
    2,2,4
    2,2,6
    1,2,5
    3,2,5
    2,1,5
    2,3,5
    

In the above example, after counting up all the sides that aren't connected to another cube, the total surface area is _`64`_.

_What is the surface area of your scanned lava droplet?_


In [2]:
from pathlib import Path
from typing import Generator

TEST = """2,2,2
1,2,2
3,2,2
2,1,2
2,3,2
2,2,1
2,2,3
2,2,4
2,2,6
1,2,5
3,2,5
2,1,5
2,3,5""".splitlines()

EXPECTED_1 = 64
EXPECTED_2 = None

DATA = Path("input/data18.txt").read_text().splitlines()


def parse(data: list[str]) -> list[tuple[int, int, int]]:
    rows = [
        tuple(int(n) for n in line.strip().split(",")) for line in data if line.strip()
    ]
    return rows


def score_1(data: list[str]) -> int:
    cubes = set(p for p in parse(data))

    common = 0
    for a, b, c in cubes:
        if (a + 1, b, c) in cubes:
            common += 1
        if (a, b + 1, c) in cubes:
            common += 1
        if (a, b, c + 1) in cubes:
            common += 1

    return 6 * len(cubes) - 2 * common


assert score_1("1,1,1\n2,1,1".splitlines()) == 10

test_score = score_1(TEST)
print("test", test_score)
assert test_score == EXPECTED_1
print("part 1", score_1(DATA))

test 64
part 1 4192


<div class="alert alert-info">Fortunately this is fairly straightforward, just count the number of neighbours that have cubes and subtract that from the total number of faces.</div>


### Part Two

Something seems off about your calculation. The cooling rate depends on exterior surface area, but your calculation also included the surface area of air pockets trapped in the lava droplet.

Instead, consider only cube sides that could be reached by the water and steam as the lava droplet tumbles into the pond. The steam will expand to reach as much as possible, completely displacing any air on the outside of the lava droplet but never expanding diagonally.

In the larger example above, exactly one cube of air is trapped within the lava droplet (at `2,2,5`), so the exterior surface area of the lava droplet is _`58`_.

_What is the exterior surface area of your scanned lava droplet?_


In [3]:

EXPECTED_2 = 58

from collections import deque


def score_2(data: list[str]) -> int:
    cubes = set(p for p in parse(data))
    min_a, max_a, min_b, max_b, min_c, max_c = (
        min(a for a, b, c in cubes) - 1,
        max(a for a, b, c in cubes) + 1,
        min(b for a, b, c in cubes) - 1,
        max(b for a, b, c in cubes) + 1,
        min(c for a, b, c in cubes) - 1,
        max(c for a, b, c in cubes) + 1,
    )
    # print(min_a, max_a, min_b, max_b, min_c, max_c)
    queue = deque([(max_a, max_b, max_c)])
    outside = {(max_a, max_b, max_c)}
    while queue:
        a, b, c = queue.popleft()
        for q in [
            (a + 1, b, c),
            (a - 1, b, c),
            (a, b + 1, c),
            (a, b - 1, c),
            (a, b, c + 1),
            (a, b, c - 1),
        ]:
            d, e, f = q
            if (
                q not in outside
                and q not in cubes
                and min_a <= d <= max_a
                and min_b <= e <= max_b
                and min_c <= f <= max_c
            ):
                outside.add(q)
                queue.append(q)

    faces = 0
    for a, b, c in cubes:
        if (a + 1, b, c) in outside:
            faces += 1
        if (a, b + 1, c) in outside:
            faces += 1
        if (a, b, c + 1) in outside:
            faces += 1
        if (a - 1, b, c) in outside:
            faces += 1
        if (a, b - 1, c) in outside:
            faces += 1
        if (a, b, c - 1) in outside:
            faces += 1

    return faces


if EXPECTED_2 is not None:
    test_score = score_2(TEST)
    print("test", test_score)
    assert test_score == EXPECTED_2
    print("part 2", score_2(DATA))


test 58
part 2 2520


<div class="alert alert-info">I took the outer bounds of the cubes and inflated the box slightly then did a flood fill from the far top corner. Then it's a case of counting how many of the neighbours to cubes were flood-filled.</div>